# TEST : compute_roughness

Here we execute similar cells as in 1_readFiles, but test out changes when executing cells twice etc.

Comparison is mostly done using 
> assert arr0 == arr0_2test
If more calculations are involved and hence small calculation differences, we use the maximal difference.

The naming of the second computed values is always appended with 
> _2test

After each comparison the _2test is deleted from the namespace.

At the end the namespace is deleted via
> %reset -f
to clear up space.

In [26]:
%load_ext autoreload
%autoreload 2

# change current working directory to parent folder, to execute script as if we execute ther
%pwd
%cd ..

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/janfelixsenge/GitBucket/github_tda_shotpeening


In [27]:
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

# import path variables
from src.config import data_path_interim, data_path_processed
# get the value for the postprocessing of the surface
from src.config import simulation_processed

#
from src.parameters.areal_surface_parameter import getRoughnessParams
from src.parameters.persistence_parameters import getPersistence, persistence_parameters
from src.postprocessing import wrapper_postprocess

In [28]:
# load data, get the according values and delete the loaded data again
data = np.load(data_path_interim / 'surface_numSimulation.npz')
height = data['values']
x_grid = data['x_grid']
y_grid = data['y_grid']

df_info = pd.read_csv(data_path_interim / 'surface_numSimulation_newcoverage.csv')

Due to the setup of the FE modelling, we have not introduced any tilting in the surface sample.

Potsprocessing therefore does not need an nominal surface to be subtracted from the surface.

Only step to be done is to apply Gaussian filtering with an cut-off of 0.08mm using FastFourier transform (with zero padding).

In [29]:
processed = simulation_processed
print('Processing of the surface is done using:', processed)

# since we already interpolated the surface we do not do that again
# during postprocessing
interpolation_size = 0

use_filter = False
removeplane = False
if 'filt' in processed:
    use_filter = True
if 'nom' in processed:
    removeplane = True
    
print('Remove a least square fitted plane from surface:', removeplane)
print('Use gausian filter:', use_filter)

Processing of the surface is done using: filt
Remove a least square fitted plane from surface: False
Use gausian filter: True


In [30]:
# postprocessing; use gaussian filter and do a least square fit
# do not interpolate the heightmap again

t0 = time.time()
znew, xnew, ynew = wrapper_postprocess(height,
                                       x_grid,
                                       y_grid,
                                       removeplane=removeplane,
                                       use_filter=use_filter,
                                       interpolation_size=interpolation_size,
                                       cut_matrix=2,
                                       dimension=2)
print(f'Postprocessing took {time.time() - t0} seconds.')

t0 = time.time()
znew_2test, xnew_2test, ynew_2test = wrapper_postprocess(height,
                                       x_grid,
                                       y_grid,
                                       removeplane=removeplane,
                                       use_filter=use_filter,
                                       interpolation_size=interpolation_size,
                                       cut_matrix=2,
                                       dimension=2)
print(f'Postprocessing took {time.time() - t0} seconds.')

Postprocessing took 67.99547982215881 seconds.


In [31]:
# check
assert (znew == znew_2test).all()
assert (xnew == xnew_2test).all()
assert (ynew == ynew_2test).all()

NameError: name 'znew_2test' is not defined

## Roughness parameters

In [32]:
# Now calculate the parameter values for the conventional parameters
t0 =time.time()

df_conventional = getRoughnessParams(znew, xnew, ynew,
                                     version='all',
                                     shift=True)
df_conventional['id'] = range(znew.shape[0])

print(f'Calculating conventional parameters took {time.time() - t0} seconds.')

# Now calculate the parameter values for the conventional parameters
t0 =time.time()

df_conventional_2test = getRoughnessParams(znew, xnew, ynew,
                                     version='all',
                                     shift=True)
df_conventional_2test['id'] = range(znew.shape[0])

print(f'Calculating conventional parameters took {time.time() - t0} seconds.')

KeyboardInterrupt: 

In [ ]:
print('The maximal difference between function values is:\n',
      np.max(df_conventional - df_conventional_2test))

The maximal difference between function values is: Sa     0.000000e+00
Sq     2.168404e-19
Ssk    1.665335e-16
Sku    0.000000e+00
Sz     0.000000e+00
Sdq    1.355253e-20
Sdr    0.000000e+00
id     0.000000e+00
dtype: float64


## PERSISTENCE

In [33]:
#now calculate the persistence parameters
t0 =time.time()

#TODO add parameter to say what we do with the infinite class
# right now: set infinite death time to maximum value of heightmap
pers = getPersistence(znew, reduced_homology=False, coeff=2)

print(f'Calculating persistence took {time.time() - t0} seconds.')
t0 =time.time()

names, params = persistence_parameters(pers)
df_persistence = pd.DataFrame(data=params, columns=names)
df_persistence['id'] = range(znew.shape[0])

print(f'Calculating persistence parameters took {time.time() - t0} seconds.')

## TEST
#now calculate the persistence parameters
t0 =time.time()

#TODO add parameter to say what we do with the infinite class
# right now: set infinite death time to maximum value of heightmap
pers_2test = getPersistence(znew, reduced_homology=False, coeff=2)

print(f'Calculating persistence took {time.time() - t0} seconds.')
t0 =time.time()

names_2test, params_2test = persistence_parameters(pers_2test)
df_persistence_2test = pd.DataFrame(data=params_2test, columns=names_2test)
df_persistence_2test['id'] = range(znew.shape[0])

print(f'Calculating persistence parameters took {time.time() - t0} seconds.')

Calculating persistence took 112.47987198829651 seconds.
Calculating persistence parameters took 11.450834035873413 seconds.
Calculating persistence took 109.93108797073364 seconds.
Calculating persistence parameters took 11.662253856658936 seconds.


In [46]:
assert (pers == pers_2test).all()

if not (df_persistence == df_persistence_2test).all().all():
    print('Maximal difference is:\n',
          np.max(np.max(df_persistence - df_persistence_2test)))
else:
    print('They are the same!')

They are the same!


## Reset complete namespace

So that the jupyter notebook doesn't clutter anything, we delete it's namespace in the end.

In [47]:
%reset 